In [127]:
import time
from collections import namedtuple

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path

## Overview

Scrapes Wall Street Journal page the list of Japanese companies

In [128]:
cwd = Path.cwd()
html_directory = cwd / 'html'

if not html_directory.exists():
    html_directory.mkdir()

### Get response and save to a file

Change headers to simulate user browsing. WSJ blocks scraping attempts from programs/scripts.

Set `refresh` to `True` to ignore cache and force retrieve the HTML content.

In [129]:
# Get the HTML and cache it in local disk
index_page_url = 'https://www.wsj.com/market-data/quotes/company-list/country/japan'
index_html_file = html_directory / 'wsj_index_page.html'

def send_request(url, output_file, refresh=False):
    # Fake headers to simulate user browsing, rather than a Python script
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }

    if refresh or not output_file.exists():
        print(f'Retrieving content from URL: {url}')
        html_content = requests.get(url, headers=headers).content
        html_content = html_content.decode()
        output_file.write_text(html_content, encoding='UTF-8')
        time.sleep(2)

send_request(index_page_url, index_html_file)

### Parse Index Page and find out the last number of the page

In [130]:
soup = BeautifulSoup(index_html_file.read_text(), 'html.parser')
print(str(soup)[:500])

<!DOCTYPE html>
 <!--TESLA DESKTOP16 V1--> <html data-region="na,us"> <head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <!--__tcfapi stub--> <!--tcfapi--> <script type="text/javascript">
!function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.


In [131]:
page_number = soup.find(class_='cl-pagination').find_all('li')[-2]
page_number = page_number.get_text()
page_number = page_number.split('-')[1]
page_number = int(page_number)
print(f'There are {page_number} page(s) to scrape')

There are 23 page(s) to scrape


### Scrape all pages and save to local disk, put some delay in between, so as to not be mistaken as a bot

In [132]:
page_output_file_list = []

for page in range(1, page_number + 1):
    page_url = f'{index_page_url}/{page}'
    page_output_file = html_directory / f'wsj{page}.html'
    page_output_file_list.append(page_output_file)
    send_request(page_url, page_output_file, refresh=False)

### Parse HTML

In [133]:
company_list = []

for html_page in page_output_file_list:
    soup = BeautifulSoup(html_page.read_text(), 'html.parser')
    table = soup.find('table', class_='cl-table')
    rows = table.find('tbody').find_all('tr')

    for r in rows:
        Company = namedtuple(
            'Company',
            [
                'CompanyNameA',
                'CompanyNameB',
                'CompanyNameC',
                'Exchange',
                'Industry',
            ]
        )
        cells = r.find_all()
        cells = [c.get_text().strip() for c in cells]
        company = Company(*cells)
        company_list.append(company._asdict())

print(len(company_list))
company_list[:5]

3322


[{'CompanyNameA': '&Do Holdings Co. Ltd. (3457)',
  'CompanyNameB': '&Do Holdings Co. Ltd. (3457)',
  'CompanyNameC': '&Do Holdings Co. Ltd.',
  'Exchange': 'XTKS',
  'Industry': 'Real Estate Developers'},
 {'CompanyNameA': '21LADY Co. Ltd. (3346)',
  'CompanyNameB': '21LADY Co. Ltd. (3346)',
  'CompanyNameC': '21LADY Co. Ltd.',
  'Exchange': 'XNGO',
  'Industry': 'Diversified Business Services'},
 {'CompanyNameA': '3-D Matrix Ltd. (7777)',
  'CompanyNameB': '3-D Matrix Ltd. (7777)',
  'CompanyNameC': '3-D Matrix Ltd.',
  'Exchange': 'XJAS',
  'Industry': 'Chemicals'},
 {'CompanyNameA': '4Cs HD Co. Ltd. (3726)',
  'CompanyNameB': '4Cs HD Co. Ltd. (3726)',
  'CompanyNameC': '4Cs HD Co. Ltd.',
  'Exchange': 'XTKS',
  'Industry': 'Specialty Retail'},
 {'CompanyNameA': '77 Bank Ltd. (8341)',
  'CompanyNameB': '77 Bank Ltd. (8341)',
  'CompanyNameC': '77 Bank Ltd.',
  'Exchange': 'XTKS',
  'Industry': 'Banking'}]

### Load the data to Pandas data frame

In [134]:
df = pd.DataFrame(company_list)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CompanyNameA  3322 non-null   object
 1   CompanyNameB  3322 non-null   object
 2   CompanyNameC  3322 non-null   object
 3   Exchange      3322 non-null   object
 4   Industry      3322 non-null   object
dtypes: object(5)
memory usage: 129.9+ KB


,CompanyNameA,CompanyNameB,CompanyNameC,Exchange,Industry
0,&Do Holdings Co. Ltd. (3457),&Do Holdings Co. Ltd. (3457),&Do Holdings Co. Ltd.,XTKS,Real Estate Developers
1,21LADY Co. Ltd. (3346),21LADY Co. Ltd. (3346),21LADY Co. Ltd.,XNGO,Diversified Business Services
2,3-D Matrix Ltd. (7777),3-D Matrix Ltd. (7777),3-D Matrix Ltd.,XJAS,Chemicals
3,4Cs HD Co. Ltd. (3726),4Cs HD Co. Ltd. (3726),4Cs HD Co. Ltd.,XTKS,Specialty Retail
4,77 Bank Ltd. (8341),77 Bank Ltd. (8341),77 Bank Ltd.,XTKS,Banking


In [135]:
df.rename(columns={
    'CompanyNameC': 'Name',
    'CompanyNameB': 'TSE',
}, inplace=True)
df = df[['Name', 'TSE', 'Exchange', 'Industry']]
df.head()

,Name,TSE,Exchange,Industry
0,&Do Holdings Co. Ltd.,&Do Holdings Co. Ltd. (3457),XTKS,Real Estate Developers
1,21LADY Co. Ltd.,21LADY Co. Ltd. (3346),XNGO,Diversified Business Services
2,3-D Matrix Ltd.,3-D Matrix Ltd. (7777),XJAS,Chemicals
3,4Cs HD Co. Ltd.,4Cs HD Co. Ltd. (3726),XTKS,Specialty Retail
4,77 Bank Ltd.,77 Bank Ltd. (8341),XTKS,Banking


In [137]:
import re

def get_symbol(value):
    symbol = re.sub(r'.*\((\d+)\)', r'\1', value)
    return symbol

df['TSE'] = df['TSE'].apply(get_symbol)
df

,Name,TSE,Exchange,Industry
0,&Do Holdings Co. Ltd.,3457,XTKS,Real Estate Developers
1,21LADY Co. Ltd.,3346,XNGO,Diversified Business Services
2,3-D Matrix Ltd.,7777,XJAS,Chemicals
3,4Cs HD Co. Ltd.,3726,XTKS,Specialty Retail
4,77 Bank Ltd.,8341,XTKS,Banking


### Check company count per industry

In [141]:
df['Industry'].value_counts()

Software                  182
Construction              137
Chemicals                 136
Real Estate Developers    122
Industrial Machinery      119
                         ... 
Insurance Brokering         1
Healthcare REITs            1
Water Utilities             1
Alternative Fuels           1
Tobacco                     1
Name: Industry, Length: 117, dtype: int64

### Export

In [142]:
df.to_csv('japanese_companies_wsj.csv', index=False, encoding='UTF-8')
print('Finished')

Finished
